In [1]:
from time import time
import os

import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras import backend as K
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
original =  pd.read_csv('fifa19.csv')
fi=original
fi=pd.DataFrame(fi)
fi = fi.drop(columns='Unnamed: 0')
fi = fi.drop(columns='ID')
fi = fi.drop(columns='Photo')
fi = fi.drop(columns='Flag')
fi = fi.drop(columns='Club Logo')
fi = fi.drop(columns='Joined')
#Correct currencies
curs=["Release Clause", "Value", "Wage"]
for cur in curs:
    
    def curr_value(x):
        x = str(x).replace('€', '')
        if('M' in str(x)):
            x = str(x).replace('M', '')
            x = float(x) * 1000000
        elif('K' in str(x)):
            x = str(x).replace('K', '')
            x = float(x) * 1000
        return float(x)
    fi[cur] = fi[cur].apply(curr_value)
   
#Correct -Dismiss + values
cols=["LS", "ST", "RS", "LW", "LF", "CF", "RF", "RW","LAM", "CAM", "RAM", "LM", "LCM", "CM", "RCM", "RM", "LWB", "LDM","CDM", "RDM", "RWB", "LB", "LCB", "CB", "RCB", "RB"]
for col in cols:
    fi[col]=fi[col].str[:-2]
    fi[col]=fi[col].astype(float)
    
#Convert contract end
fi['Contract Valid Until']=fi['Contract Valid Until'].str[-4:]
fi['Contract Valid Until']=fi['Contract Valid Until'].astype(float)
    
#Corect height values 
fi['Height']=fi['Height'].str.replace("'",'.')
fi['Height']=fi['Height'].astype(float)

#Correct Weight
fi['Weight']=fi['Weight'].str[:-3]
fi['Weight']=fi['Weight'].astype(float)

#X and y assignments
#fi=(fi[fi["Position"]!="GK"])
X = fi.loc[:, fi.columns != 'Value']
y=fi.loc[:,['Value']]
X = X.drop(columns='Name')
X = X.drop(columns='Real Face')
#identify Object columns
obj_df = X.select_dtypes(include=['object']).copy()
obj_df.head()

,Value
0,110500000.0
1,77000000.0
2,118500000.0
3,72000000.0
4,102000000.0
5,93000000.0
6,67000000.0
7,80000000.0
8,51000000.0
9,68000000.0


In [3]:
X_dum = pd.get_dummies(X[obj_df.columns], dummy_na=True,drop_first=True)
X = pd.concat([X.drop(obj_df.columns, axis=1), pd.get_dummies(X[obj_df.columns])], axis=1)
#See Correlations & Drop highly correlated attributes
X1 = pd.DataFrame(X)
corr = X1.corr()
corr_matrix = X1.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
X=X.drop(columns=to_drop, axis=1)
columnlist=X.columns.tolist()
# Taking care of missing data
from sklearn.preprocessing import Imputer
# imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
imputer = Imputer()
imputer = imputer.fit(X)
X_m = imputer.transform(X)
X=pd.DataFrame(X_m)

# from sklearn.preprocessing import Imputer
# imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
imputer = Imputer()
print('the y:\n', y)
imputer = imputer.fit(y)
y = imputer.transform(y)
y=pd.DataFrame(y)

# from sklearn.preprocessing import Imputer
# imputer = Imputer(missing_values = 0, strategy = 'mean', axis = 0)
imputer = Imputer()
imputer = imputer.fit(y)
y = imputer.transform(y)
y=pd.DataFrame(y)

#Split Dataset Test vs. Train

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

/home/paologio/anaconda3/envs/venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


the y:
              Value
0      110500000.0
1       77000000.0
2      118500000.0
3       72000000.0
4      102000000.0
5       93000000.0
6       67000000.0
7       80000000.0
8       51000000.0
9       68000000.0
10      77000000.0
11      76500000.0
12      44000000.0
13      60000000.0
14      63000000.0
15      89000000.0
16      83500000.0
17      78000000.0
18      58000000.0
19      53500000.0
20      51500000.0
21      60000000.0
22      38000000.0
23      64500000.0
24      27000000.0
25      81000000.0
26      69500000.0
27      59500000.0
28      69500000.0
29      62000000.0
...            ...
18177      70000.0
18178      50000.0
18179      50000.0
18180      40000.0
18181      50000.0
18182      50000.0
18183          0.0
18184      40000.0
18185      50000.0
18186      60000.0
18187      40000.0
18188      60000.0
18189      60000.0
18190      60000.0
18191      50000.0
18192      50000.0
18193      50000.0
18194      50000.0
18195      60000.0
18196      60000.0
1819

/home/paologio/anaconda3/envs/venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/paologio/anaconda3/envs/venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def coeff_determination(y_test, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_test-y_pred ))
    SS_tot = K.sum(K.square( y_test - K.mean(y_test) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()))

In [5]:
regressor = Sequential()
# Adding the input layer and the first hidden layer
regressor.add(Dense(output_dim = 624, kernel_initializer='normal', activation = 'relu', input_dim = 1248))
# Adding the second hidden layer
regressor.add(Dense(output_dim = 300, kernel_initializer='normal', activation = 'relu'))
# Adding the 3rd hidden layer
regressor.add(Dense(output_dim = 150, kernel_initializer='normal', activation = 'relu'))
# Adding the 4th hidden layer
regressor.add(Dense(output_dim = 75, kernel_initializer='normal', activation = 'relu'))
# Adding the output layer
regressor.add(Dense(output_dim = 1, kernel_initializer='normal', activation = 'linear'))

Instructions for updating:
Colocations handled automatically by placer.


/home/paologio/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="normal", activation="relu", input_dim=1248, units=624)`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/paologio/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="normal", activation="relu", units=300)`
  """
/home/paologio/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="normal", activation="relu", units=150)`
  import sys
/home/paologio/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="normal", activation="relu", units=75)`
  if __name__ == '__main__':
/ho

In [6]:
ts_board = TensorBoard(log_dir='wage_logs/{}'.format(time()))

In [7]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = [coeff_determination])
# Fitting the ANN to the Training set-I have tried it with different batch sizes and epochs but it was overfitted
regressor.fit(X_train, y_train, batch_size = 950, nb_epoch = 7, callbacks=[ts_board])

Instructions for updating:
Use tf.cast instead.


/home/paologio/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/7
14565/14565 [==============================] - 2s 110us/step - loss: 18737862412714.7539 - coeff_determination: 0.4055
Epoch 2/7
14565/14565 [==============================] - 1s 101us/step - loss: 4083906607138.4492 - coeff_determination: 0.8675
Epoch 3/7
14565/14565 [==============================] - 1s 100us/step - loss: 2231639039429.1191 - coeff_determination: 0.9324
Epoch 4/7
14565/14565 [==============================] - 1s 99us/step - loss: 2002009526810.5403 - coeff_determination: 0.9322
Epoch 5/7
14565/14565 [==============================] - 1s 100us/step - loss: 1936577754270.5391 - coeff_determination: 0.9382
Epoch 6/7
14565/14565 [==============================] - 1s 99us/step - loss: 1914686798477.1384 - coeff_determination: 0.9327
Epoch 7/7
14565/14565 [==============================] - 1s 99us/step - loss: 1920374489014.8823 - coeff_determination: 0.9355


In [8]:
def mean_abs_error(prediction, target):
    res = 0
    tot = 0
    for i in range(len(target)):
        if target[i][0] != 0 and not np.isnan(prediction[i][0]):
#             print(prediction[i][0], target[i][0])
            res += abs(prediction[i][0] - target[i][0])
            tot += 1
    print(tot, len(target), len(prediction))
    return round(res / tot, 2)

In [14]:
res = regressor.predict(X_test)
# for i in range(len(res)): print(res[i])
mean_abs_error(res, y_test.values), max(y_test.values)

3598 3642 3642


(383625.22, array([1.105e+08]))